In [1]:
import os
import json
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from torch import nn, optim
from torchvision import models
from PIL import Image
import albumentations as A
from albumentations.pytorch import ToTensorV2
import cv2

# Rutas
train_img_dir = r'C:\CatFLW dataset\train\images'
train_labels_dir = r'C:\CatFLW dataset\train\labels'
val_img_dir = r'C:\CatFLW dataset\val\images'
val_labels_dir = r'C:\CatFLW dataset\val\labels'

# Definir los índices de las regiones según los puntos proporcionados
ojo_der = [3, 4, 5, 6, 7, 36, 37, 38]
ojo_izq = [1, 8, 9, 10, 11, 39, 40, 41]
oreja_izq = [27, 28, 29, 30, 31]

# Función para calcular el punto promedio de una región
def calcular_centro_region(landmarks, indices):
    puntos = [landmarks[i] for i in indices]
    centro = np.mean(puntos, axis=0)
    return centro

# Dataset personalizado
class EarLeftDataset(Dataset):
    def __init__(self, img_dir, labels_dir, transform=None):
        self.img_dir = img_dir
        self.labels_dir = labels_dir
        self.transform = transform
        self.image_files = [f for f in os.listdir(img_dir) if f.endswith('.png')]

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_name = self.image_files[idx]
        img_path = os.path.join(self.img_dir, img_name)
        label_path = os.path.join(self.labels_dir, img_name.replace('.png', '.json'))
        
        # Cargar imagen
        image = np.array(Image.open(img_path).convert('RGB'))
        
        # Cargar etiquetas y puntos de referencia
        with open(label_path, 'r') as f:
            labels = json.load(f)
            landmarks = np.array(labels['labels'])  # Convertir a array de NumPy
            bounding_box = labels['bounding_boxes']
        
        # Recortar la imagen usando el bounding box
        x_min, y_min, x_max, y_max = map(int, bounding_box)
        image_cropped = image[y_min:y_max, x_min:x_max]
        
        # Calcular dimensiones originales
        original_height, original_width = image_cropped.shape[:2]
        
        # Calcular centros de los ojos
        centro_ojo_der = calcular_centro_region(landmarks, ojo_der)
        centro_ojo_izq = calcular_centro_region(landmarks, ojo_izq)
        centro_oreja_izq = calcular_centro_region(landmarks, oreja_izq)
        
        # Ajustar coordenadas de los centros al recorte
        centro_ojo_der -= np.array([x_min, y_min])
        centro_ojo_izq -= np.array([x_min, y_min])
        centro_oreja_izq -= np.array([x_min, y_min])
        
        # Calcular el ángulo de rotación
        dx = centro_ojo_izq[0] - centro_ojo_der[0]
        dy = centro_ojo_izq[1] - centro_ojo_der[1]
        angle = np.degrees(np.arctan2(dy, dx))
        
        # Crear matriz de rotación
        eye_center = ((centro_ojo_der[0] + centro_ojo_izq[0]) // 2,
                      (centro_ojo_der[1] + centro_ojo_izq[1]) // 2)
        rotation_matrix = cv2.getRotationMatrix2D(eye_center, angle, 1.0)
        
        # Rotar la imagen
        image_aligned = cv2.warpAffine(
            image_cropped, rotation_matrix, (original_width, original_height), flags=cv2.INTER_LINEAR)
        
        # Convertir los puntos clave de la oreja izquierda
        ear_left_points = np.array([landmarks[i] for i in oreja_izq], dtype=np.float32)
        ear_left_points -= [x_min, y_min]

        # Transformar los puntos clave con la misma rotación
        ones = np.ones(shape=(len(ear_left_points), 1))
        points_ones = np.hstack([ear_left_points, ones])
        ear_left_points_rotated = rotation_matrix.dot(points_ones.T).T
        
        # **Nuevo**: Recorte alrededor del punto centrado de la oreja izquierda
        centro_x, centro_y = calcular_centro_region(ear_left_points_rotated, range(len(oreja_izq)))

        # Definir los límites del recorte de 112x112 píxeles
        half_crop_size = 56
        x1 = int(max(0, centro_x - half_crop_size))
        y1 = int(max(0, centro_y - half_crop_size))
        x2 = int(min(original_width, centro_x + half_crop_size))
        y2 = int(min(original_height, centro_y + half_crop_size))
        
        # Recortar la imagen
        image_cropped_ear = image_aligned[y1:y2, x1:x2]
        
        # Redimensionar a 224x224
        image_resized = cv2.resize(image_cropped_ear, (224, 224))

        # Ajustar los puntos faciales de la oreja izquierda al nuevo recorte
        ear_left_points_adjusted = [(p[0] - x1, p[1] - y1) for p in ear_left_points_rotated]

        # Redimensionar las coordenadas de los puntos a la escala 224x224
        scale_x = 224 / (x2 - x1)
        scale_y = 224 / (y2 - y1)
        ear_left_points_scaled = [(p[0] * scale_x, p[1] * scale_y) for p in ear_left_points_adjusted]

        # Aplicar transformaciones de imagen y etiquetas
        if self.transform:
            augmented = self.transform(image=image_resized, keypoints=ear_left_points_scaled)
            image = augmented['image']
            ear_left_points = np.array(augmented['keypoints'], dtype=np.float32).flatten()
        
        return image, torch.tensor(ear_left_points).float(), img_name, original_width, original_height

# Transformaciones de aumento de datos para el conjunto de entrenamiento
train_transforms = A.Compose([
    A.Rotate(limit=30, p=0.5),  # Rotación aleatoria con 50% de probabilidad
    A.ColorJitter(p=0.5),  # Cambio en el balance de color, brillo, contraste y nitidez con 50% de probabilidad
    A.RandomBrightnessContrast(p=0.5),  # Cambio en el brillo y contraste con 50% de probabilidad
    A.GaussianBlur(p=0.1),  # Aplicación de máscaras de desenfoque con 50% de probabilidad
    A.GaussNoise(p=0.1),  # Ruido aleatorio con 50% de probabilidad
    A.Resize(224, 224),  # Redimensionar a 224x224
    A.Normalize(),
    ToTensorV2(),
], keypoint_params=A.KeypointParams(format='xy', remove_invisible=False))

# Transformaciones para el conjunto de validación
val_transforms = A.Compose([
    A.Resize(224, 224),  # Redimensionar a 224x224
    A.Normalize(),
    ToTensorV2(),
], keypoint_params=A.KeypointParams(format='xy', remove_invisible=False))

# Crear los datasets de entrenamiento y validación
train_dataset = EarLeftDataset(train_img_dir, train_labels_dir, transform=train_transforms)
val_dataset = EarLeftDataset(val_img_dir, val_labels_dir, transform=val_transforms)

# Crear DataLoaders
train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=16, shuffle=False)

# Modelo basado en DenseNet-121
class DenseNet121EarLeft(nn.Module):
    def __init__(self):
        super(DenseNet121EarLeft, self).__init__()
        self.backbone = models.densenet121(pretrained=True)
        self.backbone.classifier = nn.Identity()
        self.fc1 = nn.Linear(1024, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, len(oreja_izq) * 2)
        
    def forward(self, x):
        x = self.backbone(x)
        x = nn.ReLU()(self.fc1(x))
        x = nn.ReLU()(self.fc2(x))
        return self.fc3(x)

# Configuración y entrenamiento
model = DenseNet121EarLeft().cuda()
optimizer = optim.AdamW(model.parameters(), lr=1e-4)
criterion = nn.MSELoss()
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=75, verbose=True)

def calculate_nme(predictions, ground_truths, num_landmarks):
    """
    Calcula el NME (Normalized Mean Error) utilizando la fórmula específica.
    
    :param predictions: Array de predicciones, de tamaño (N, M, 2), donde
                        N es el número de imágenes, M el número de landmarks, 
                        y 2 son las coordenadas (x, y) de cada punto.
    :param ground_truths: Array de valores reales (ground truths), del mismo tamaño que predictions.
    :param num_landmarks: Número de landmarks en cada imagen (M).
    :return: NME calculado sobre todo el dataset.
    """
    num_images = predictions.shape[0]
    total_error = 0.0

    for i in range(num_images):
        # Verificar que los índices utilizados para calcular la distancia interocular sean correctos
        if len(ground_truths[i]) < 5:
            raise ValueError(f"Expected at least 5 landmarks, but got {len(ground_truths[i])}")

        # Calcula la distancia interocular usando los puntos 0 y 4 (ajustado para oreja izquierda)
        iodi = np.linalg.norm(ground_truths[i, 0] - ground_truths[i, 4])
        
        # Suma de errores normalizados por imagen
        for j in range(num_landmarks):
            pred_coord = predictions[i, j]
            gt_coord = ground_truths[i, j]
            error = np.linalg.norm(pred_coord - gt_coord) / iodi
            total_error += error
    
    # Aplicar la fórmula para el NME
    nme = total_error / (num_landmarks * num_images)
    return nme


def train_model(model, train_dataloader, optimizer, criterion, num_epochs=300, val_dataloader=None, scheduler=None):
    best_loss = float('inf')
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for images, ear_left_points, _, _, _ in train_dataloader:
            images, ear_left_points = images.cuda(), ear_left_points.cuda()
            outputs = model(images)
            loss = criterion(outputs, ear_left_points)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        
        epoch_loss = running_loss / len(train_dataloader)
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}')
        
        # Validación y cálculo del NME
        if val_dataloader:
            model.eval()
            val_loss = 0.0
            all_labels = []
            all_predictions = []
            with torch.no_grad():
                for val_images, val_ear_left_points, _, _, _ in val_dataloader:
                    val_images, val_ear_left_points = val_images.cuda(), val_ear_left_points.cuda()
                    val_outputs = model(val_images)
                    val_loss += criterion(val_outputs, val_ear_left_points).item()
                    
                    # Guardar las predicciones y etiquetas reales para el cálculo de NME
                    all_labels.append(val_ear_left_points.cpu().numpy().reshape(-1, len(oreja_izq), 2))
                    all_predictions.append(val_outputs.cpu().numpy().reshape(-1, len(oreja_izq), 2))

            val_loss /= len(val_dataloader)
            print(f'Validation Loss: {val_loss:.4f}')
            
            # Concatenar todas las predicciones y *ground truths* para el NME
            all_labels = np.concatenate(all_labels, axis=0)
            all_predictions = np.concatenate(all_predictions, axis=0)
            nme = calculate_nme(all_predictions, all_labels, num_landmarks=len(oreja_izq))
            print(f'NME: {nme:.4f}')
            
            # Guardar el mejor modelo
            if val_loss < best_loss:
                best_loss = val_loss
                torch.save(model.state_dict(), 'EarLeftLandmarks V4.pth')
                print(f'Best model saved at epoch {epoch+1} with val_loss {best_loss:.4f}')
                
            if scheduler:
                scheduler.step(val_loss)

    print("Training complete.")

# Entrenar el modelo
train_model(model, train_dataloader, optimizer, criterion, num_epochs=300, val_dataloader=val_dataloader, scheduler=scheduler)

c:\Users\luisb250\AppData\Local\Programs\Python\Python311\Lib\site-packages\albumentations\__init__.py:13: UserWarning: A new version of Albumentations is available: 1.4.24 (you have 1.4.18). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()
c:\Users\luisb250\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\luisb250\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet121_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet121_Weights.DEFAULT` 

Epoch [1/300], Loss: 17982.1961
Validation Loss: 15987.8258
NME: 1.4587
Best model saved at epoch 1 with val_loss 15987.8258
Epoch [2/300], Loss: 10561.6879
Validation Loss: 4950.5171
NME: 0.7254
Best model saved at epoch 2 with val_loss 4950.5171
Epoch [3/300], Loss: 2990.6039
Validation Loss: 1932.6013
NME: 0.4025
Best model saved at epoch 3 with val_loss 1932.6013
Epoch [4/300], Loss: 1830.6859
Validation Loss: 1454.0242
NME: 0.3464
Best model saved at epoch 4 with val_loss 1454.0242
Epoch [5/300], Loss: 1629.3723
Validation Loss: 1387.1694
NME: 0.3458
Best model saved at epoch 5 with val_loss 1387.1694
Epoch [6/300], Loss: 1585.8330
Validation Loss: 1382.3968
NME: 0.3371
Best model saved at epoch 6 with val_loss 1382.3968
Epoch [7/300], Loss: 1554.8872
Validation Loss: 1365.4486
NME: 0.3414
Best model saved at epoch 7 with val_loss 1365.4486
Epoch [8/300], Loss: 1545.8507
Validation Loss: 1329.1997
NME: 0.3398
Best model saved at epoch 8 with val_loss 1329.1997
Epoch [9/300], Loss:

In [2]:
import os
import json
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from torch import nn, optim
from torchvision import models
from PIL import Image
import albumentations as A
from albumentations.pytorch import ToTensorV2
import cv2

# Rutas
train_img_dir = r'C:\CatFLW dataset\train\images'
train_labels_dir = r'C:\CatFLW dataset\train\labels'
val_img_dir = r'C:\CatFLW dataset\val\images'
val_labels_dir = r'C:\CatFLW dataset\val\labels'

# Definir los índices de las regiones según los puntos proporcionados
ojo_der = [3, 4, 5, 6, 7, 36, 37, 38]
ojo_izq = [1, 8, 9, 10, 11, 39, 40, 41]
oreja_der = [22, 23, 24, 25, 26]  # Cambiado a oreja derecha

# Función para calcular el punto promedio de una región
def calcular_centro_region(landmarks, indices):
    puntos = [landmarks[i] for i in indices]
    centro = np.mean(puntos, axis=0)
    return centro

# Dataset personalizado
class EarRightDataset(Dataset):
    def __init__(self, img_dir, labels_dir, transform=None):
        self.img_dir = img_dir
        self.labels_dir = labels_dir
        self.transform = transform
        self.image_files = [f for f in os.listdir(img_dir) if f.endswith('.png')]

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_name = self.image_files[idx]
        img_path = os.path.join(self.img_dir, img_name)
        label_path = os.path.join(self.labels_dir, img_name.replace('.png', '.json'))
        
        # Cargar imagen
        image = np.array(Image.open(img_path).convert('RGB'))
        
        # Cargar etiquetas y puntos de referencia
        with open(label_path, 'r') as f:
            labels = json.load(f)
            landmarks = np.array(labels['labels'])  # Convertir a array de NumPy
            bounding_box = labels['bounding_boxes']
        
        # Recortar la imagen usando el bounding box
        x_min, y_min, x_max, y_max = map(int, bounding_box)
        image_cropped = image[y_min:y_max, x_min:x_max]
        
        # Calcular dimensiones originales
        original_height, original_width = image_cropped.shape[:2]
        
        # Calcular centros de los ojos
        centro_ojo_der = calcular_centro_region(landmarks, ojo_der)
        centro_ojo_izq = calcular_centro_region(landmarks, ojo_izq)
        centro_oreja_der = calcular_centro_region(landmarks, oreja_der)  # Cambiado a oreja derecha
        
        # Ajustar coordenadas de los centros al recorte
        centro_ojo_der -= np.array([x_min, y_min])
        centro_ojo_izq -= np.array([x_min, y_min])
        centro_oreja_der -= np.array([x_min, y_min])  # Cambiado a oreja derecha
        
        # Calcular el ángulo de rotación
        dx = centro_ojo_izq[0] - centro_ojo_der[0]
        dy = centro_ojo_izq[1] - centro_ojo_der[1]
        angle = np.degrees(np.arctan2(dy, dx))
        
        # Crear matriz de rotación
        eye_center = ((centro_ojo_der[0] + centro_ojo_izq[0]) // 2,
                      (centro_ojo_der[1] + centro_ojo_izq[1]) // 2)
        rotation_matrix = cv2.getRotationMatrix2D(eye_center, angle, 1.0)
        
        # Rotar la imagen
        image_aligned = cv2.warpAffine(
            image_cropped, rotation_matrix, (original_width, original_height), flags=cv2.INTER_LINEAR)
        
        # Convertir los puntos clave de la oreja derecha
        ear_right_points = np.array([landmarks[i] for i in oreja_der], dtype=np.float32)  # Cambiado a oreja derecha
        ear_right_points -= [x_min, y_min]

        # Transformar los puntos clave con la misma rotación
        ones = np.ones(shape=(len(ear_right_points), 1))
        points_ones = np.hstack([ear_right_points, ones])
        ear_right_points_rotated = rotation_matrix.dot(points_ones.T).T
        
        # **Nuevo**: Recorte alrededor del punto centrado de la oreja derecha
        centro_x, centro_y = calcular_centro_region(ear_right_points_rotated, range(len(oreja_der)))  # Cambiado a oreja derecha

        # Definir los límites del recorte de 112x112 píxeles
        half_crop_size = 56
        x1 = int(max(0, centro_x - half_crop_size))
        y1 = int(max(0, centro_y - half_crop_size))
        x2 = int(min(original_width, centro_x + half_crop_size))
        y2 = int(min(original_height, centro_y + half_crop_size))
        
        # Recortar la imagen
        image_cropped_ear = image_aligned[y1:y2, x1:x2]
        
        # Redimensionar a 224x224
        image_resized = cv2.resize(image_cropped_ear, (224, 224))

        # Ajustar los puntos faciales de la oreja derecha al nuevo recorte
        ear_right_points_adjusted = [(p[0] - x1, p[1] - y1) for p in ear_right_points_rotated]

        # Redimensionar las coordenadas de los puntos a la escala 224x224
        scale_x = 224 / (x2 - x1)
        scale_y = 224 / (y2 - y1)
        ear_right_points_scaled = [(p[0] * scale_x, p[1] * scale_y) for p in ear_right_points_adjusted]

        # Aplicar transformaciones de imagen y etiquetas
        if self.transform:
            augmented = self.transform(image=image_resized, keypoints=ear_right_points_scaled)
            image = augmented['image']
            ear_right_points = np.array(augmented['keypoints'], dtype=np.float32).flatten()
        
        return image, torch.tensor(ear_right_points).float(), img_name, original_width, original_height

# Transformaciones de aumento de datos para el conjunto de entrenamiento
train_transforms = A.Compose([
    A.Rotate(limit=30, p=0.5),  # Rotación aleatoria con 50% de probabilidad
    A.ColorJitter(p=0.5),  # Cambio en el balance de color, brillo, contraste y nitidez con 50% de probabilidad
    A.RandomBrightnessContrast(p=0.5),  # Cambio en el brillo y contraste con 50% de probabilidad
    A.GaussianBlur(p=0.1),  # Aplicación de máscaras de desenfoque con 50% de probabilidad
    A.GaussNoise(p=0.1),  # Ruido aleatorio con 50% de probabilidad
    A.Resize(224, 224),  # Redimensionar a 224x224
    A.Normalize(),
    ToTensorV2(),
], keypoint_params=A.KeypointParams(format='xy', remove_invisible=False))

# Transformaciones para el conjunto de validación
val_transforms = A.Compose([
    A.Resize(224, 224),  # Redimensionar a 224x224
    A.Normalize(),
    ToTensorV2(),
], keypoint_params=A.KeypointParams(format='xy', remove_invisible=False))

# Crear los datasets de entrenamiento y validación
train_dataset = EarRightDataset(train_img_dir, train_labels_dir, transform=train_transforms)
val_dataset = EarRightDataset(val_img_dir, val_labels_dir, transform=val_transforms)

# Crear DataLoaders
train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=16, shuffle=False)

# Modelo basado en DenseNet-121
class DenseNet121EarRight(nn.Module):
    def __init__(self):
        super(DenseNet121EarRight, self).__init__()
        self.backbone = models.densenet121(pretrained=True)
        self.backbone.classifier = nn.Identity()
        self.fc1 = nn.Linear(1024, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, len(oreja_der) * 2)  # Cambiado a oreja derecha
        
    def forward(self, x):
        x = self.backbone(x)
        x = nn.ReLU()(self.fc1(x))
        x = nn.ReLU()(self.fc2(x))
        return self.fc3(x)

# Configuración y entrenamiento
model = DenseNet121EarRight().cuda()
optimizer = optim.AdamW(model.parameters(), lr=1e-4)
criterion = nn.MSELoss()
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=75, verbose=True)

def calculate_nme(predictions, ground_truths, num_landmarks):
    """
    Calcula el NME (Normalized Mean Error) utilizando la fórmula específica.
    
    :param predictions: Array de predicciones, de tamaño (N, M, 2), donde
                        N es el número de imágenes, M el número de landmarks, 
                        y 2 son las coordenadas (x, y) de cada punto.
    :param ground_truths: Array de valores reales (ground truths), del mismo tamaño que predictions.
    :param num_landmarks: Número de landmarks en cada imagen (M).
    :return: NME calculado sobre todo el dataset.
    """
    num_images = predictions.shape[0]
    total_error = 0.0

    for i in range(num_images):
        # Verificar que los índices utilizados para calcular la distancia interocular sean correctos
        if len(ground_truths[i]) < 5:
            raise ValueError(f"Expected at least 5 landmarks, but got {len(ground_truths[i])}")

        # Calcula la distancia interocular usando los puntos 0 y 4 (ajustado para oreja derecha)
        iodi = np.linalg.norm(ground_truths[i, 0] - ground_truths[i, 4])
        
        # Suma de errores normalizados por imagen
        for j in range(num_landmarks):
            pred_coord = predictions[i, j]
            gt_coord = ground_truths[i, j]
            error = np.linalg.norm(pred_coord - gt_coord) / iodi
            total_error += error
    
    # Aplicar la fórmula para el NME
    nme = total_error / (num_landmarks * num_images)
    return nme


def train_model(model, train_dataloader, optimizer, criterion, num_epochs=300, val_dataloader=None, scheduler=None):
    best_loss = float('inf')
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for images, ear_right_points, _, _, _ in train_dataloader:  # Cambiado a oreja derecha
            images, ear_right_points = images.cuda(), ear_right_points.cuda()
            outputs = model(images)
            loss = criterion(outputs, ear_right_points)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        
        epoch_loss = running_loss / len(train_dataloader)
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}')
        
        # Validación y cálculo del NME
        if val_dataloader:
            model.eval()
            val_loss = 0.0
            all_labels = []
            all_predictions = []
            with torch.no_grad():
                for val_images, val_ear_right_points, _, _, _ in val_dataloader:  # Cambiado a oreja derecha
                    val_images, val_ear_right_points = val_images.cuda(), val_ear_right_points.cuda()
                    val_outputs = model(val_images)
                    val_loss += criterion(val_outputs, val_ear_right_points).item()
                    
                    # Guardar las predicciones y etiquetas reales para el cálculo de NME
                    all_labels.append(val_ear_right_points.cpu().numpy().reshape(-1, len(oreja_der), 2))  # Cambiado a oreja derecha
                    all_predictions.append(val_outputs.cpu().numpy().reshape(-1, len(oreja_der), 2))  # Cambiado a oreja derecha

            val_loss /= len(val_dataloader)
            print(f'Validation Loss: {val_loss:.4f}')
            
            # Concatenar todas las predicciones y *ground truths* para el NME
            all_labels = np.concatenate(all_labels, axis=0)
            all_predictions = np.concatenate(all_predictions, axis=0)
            nme = calculate_nme(all_predictions, all_labels, num_landmarks=len(oreja_der))  # Cambiado a oreja derecha
            print(f'NME: {nme:.4f}')
            
            # Guardar el mejor modelo
            if val_loss < best_loss:
                best_loss = val_loss
                torch.save(model.state_dict(), 'EarRightLandmarks V4.pth')  # Cambiado a oreja derecha
                print(f'Best model saved at epoch {epoch+1} with val_loss {best_loss:.4f}')
                
            if scheduler:
                scheduler.step(val_loss)

    print("Training complete.")

# Entrenar el modelo
train_model(model, train_dataloader, optimizer, criterion, num_epochs=300, val_dataloader=val_dataloader, scheduler=scheduler)

c:\Users\luisb250\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\luisb250\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet121_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet121_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
c:\Users\luisb250\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\optim\lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Epoch [1/300], Loss: 15523.8694
Validation Loss: 13076.5193
NME: 1.1804
Best model saved at epoch 1 with val_loss 13076.5193
Epoch [2/300], Loss: 7981.0826
Validation Loss: 2812.2768
NME: 0.4925
Best model saved at epoch 2 with val_loss 2812.2768
Epoch [3/300], Loss: 1751.7138
Validation Loss: 1183.1187
NME: 0.2623
Best model saved at epoch 3 with val_loss 1183.1187
Epoch [4/300], Loss: 1340.1935
Validation Loss: 1143.6206
NME: 0.2696
Best model saved at epoch 4 with val_loss 1143.6206
Epoch [5/300], Loss: 1315.2226
Validation Loss: 1111.7539
NME: 0.2731
Best model saved at epoch 5 with val_loss 1111.7539
Epoch [6/300], Loss: 1287.6791
Validation Loss: 1119.6959
NME: 0.2646
Epoch [7/300], Loss: 1273.9931
Validation Loss: 1100.1614
NME: 0.2676
Best model saved at epoch 7 with val_loss 1100.1614
Epoch [8/300], Loss: 1260.6853
Validation Loss: 1101.2115
NME: 0.2600
Epoch [9/300], Loss: 1244.1019
Validation Loss: 1089.7598
NME: 0.2630
Best model saved at epoch 9 with val_loss 1089.7598
Epo

In [3]:
import os
import json
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from torch import nn, optim
from torchvision import models
from PIL import Image
import albumentations as A
from albumentations.pytorch import ToTensorV2
import cv2

# Rutas
train_img_dir = r'C:\CatFLW dataset\train\images'
train_labels_dir = r'C:\CatFLW dataset\train\labels'
val_img_dir = r'C:\CatFLW dataset\val\images'
val_labels_dir = r'C:\CatFLW dataset\val\labels'

# Definir los índices de las regiones según los puntos proporcionados
ojo_der = [3, 4, 5, 6, 7, 36, 37, 38]
ojo_izq = [1, 8, 9, 10, 11, 39, 40, 41]

# Función para calcular el punto promedio de una región
def calcular_centro_region(landmarks, indices):
    puntos = [landmarks[i] for i in indices]
    centro = np.mean(puntos, axis=0)
    return centro

# Dataset personalizado
class EyeLeftDataset(Dataset):
    def __init__(self, img_dir, labels_dir, transform=None):
        self.img_dir = img_dir
        self.labels_dir = labels_dir
        self.transform = transform
        self.image_files = [f for f in os.listdir(img_dir) if f.endswith('.png')]

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_name = self.image_files[idx]
        img_path = os.path.join(self.img_dir, img_name)
        label_path = os.path.join(self.labels_dir, img_name.replace('.png', '.json'))
        
        # Cargar imagen
        image = np.array(Image.open(img_path).convert('RGB'))
        
        # Cargar etiquetas y puntos de referencia
        with open(label_path, 'r') as f:
            labels = json.load(f)
            landmarks = np.array(labels['labels'])  # Convertir a array de NumPy
            bounding_box = labels['bounding_boxes']
        
        # Recortar la imagen usando el bounding box
        x_min, y_min, x_max, y_max = map(int, bounding_box)
        image_cropped = image[y_min:y_max, x_min:x_max]
        
        # Calcular dimensiones originales
        original_height, original_width = image_cropped.shape[:2]
        
        # Calcular centros de los ojos
        centro_ojo_der = calcular_centro_region(landmarks, ojo_der)
        centro_ojo_izq = calcular_centro_region(landmarks, ojo_izq)
        
        # Ajustar coordenadas de los centros al recorte
        centro_ojo_der -= np.array([x_min, y_min])
        centro_ojo_izq -= np.array([x_min, y_min])
        
        # Calcular el ángulo de rotación
        dx = centro_ojo_izq[0] - centro_ojo_der[0]
        dy = centro_ojo_izq[1] - centro_ojo_der[1]
        angle = np.degrees(np.arctan2(dy, dx))
        
        # Crear matriz de rotación
        eye_center = ((centro_ojo_der[0] + centro_ojo_izq[0]) // 2,
                      (centro_ojo_der[1] + centro_ojo_izq[1]) // 2)
        rotation_matrix = cv2.getRotationMatrix2D(eye_center, angle, 1.0)
        
        # Rotar la imagen
        image_aligned = cv2.warpAffine(
            image_cropped, rotation_matrix, (original_width, original_height), flags=cv2.INTER_LINEAR)
        
        # Convertir los puntos clave del ojo izquierdo
        eye_left_points = np.array([landmarks[i] for i in ojo_izq], dtype=np.float32)
        eye_left_points -= [x_min, y_min]

        # Transformar los puntos clave con la misma rotación
        ones = np.ones(shape=(len(eye_left_points), 1))
        points_ones = np.hstack([eye_left_points, ones])
        eye_left_points_rotated = rotation_matrix.dot(points_ones.T).T
        
        # **Nuevo**: Recorte alrededor del punto centrado del ojo izquierdo
        centro_x, centro_y = calcular_centro_region(eye_left_points_rotated, range(len(ojo_izq)))

        # Definir los límites del recorte de 56x56 píxeles
        half_crop_size = 28
        x1 = int(max(0, centro_x - half_crop_size))
        y1 = int(max(0, centro_y - half_crop_size))
        x2 = int(min(original_width, centro_x + half_crop_size))
        y2 = int(min(original_height, centro_y + half_crop_size))
        
        # Recortar la imagen
        image_cropped_eye = image_aligned[y1:y2, x1:x2]
        
        # Redimensionar a 224x224
        image_resized = cv2.resize(image_cropped_eye, (224, 224))

        # Ajustar los puntos faciales del ojo izquierdo al nuevo recorte
        eye_left_points_adjusted = [(p[0] - x1, p[1] - y1) for p in eye_left_points_rotated]

        # Redimensionar las coordenadas de los puntos a la escala 224x224
        scale_x = 224 / (x2 - x1)
        scale_y = 224 / (y2 - y1)
        eye_left_points_scaled = [(p[0] * scale_x, p[1] * scale_y) for p in eye_left_points_adjusted]

        # Aplicar transformaciones de imagen y etiquetas
        if self.transform:
            augmented = self.transform(image=image_resized, keypoints=eye_left_points_scaled)
            image = augmented['image']
            eye_left_points = np.array(augmented['keypoints'], dtype=np.float32).flatten()
        
        return image, torch.tensor(eye_left_points).float(), img_name, original_width, original_height

# Transformaciones de aumento de datos para el conjunto de entrenamiento
train_transforms = A.Compose([
    A.Rotate(limit=30, p=0.5),  # Rotación aleatoria con 50% de probabilidad
    A.ColorJitter(p=0.5),  # Cambio en el balance de color, brillo, contraste y nitidez con 50% de probabilidad
    A.RandomBrightnessContrast(p=0.5),  # Cambio en el brillo y contraste con 50% de probabilidad
    A.GaussianBlur(p=0.1),  # Aplicación de máscaras de desenfoque con 50% de probabilidad
    A.GaussNoise(p=0.1),  # Ruido aleatorio con 50% de probabilidad
    A.Resize(224, 224),  # Redimensionar a 224x224
    A.Normalize(),
    ToTensorV2(),
], keypoint_params=A.KeypointParams(format='xy', remove_invisible=False))

# Transformaciones para el conjunto de validación
val_transforms = A.Compose([
    A.Resize(224, 224),  # Redimensionar a 224x224
    A.Normalize(),
    ToTensorV2(),
], keypoint_params=A.KeypointParams(format='xy', remove_invisible=False))

# Crear los datasets de entrenamiento y validación
train_dataset = EyeLeftDataset(train_img_dir, train_labels_dir, transform=train_transforms)
val_dataset = EyeLeftDataset(val_img_dir, val_labels_dir, transform=val_transforms)

# Crear DataLoaders
train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=16, shuffle=False)

# Modelo basado en DenseNet-121
class DenseNet121EyeLeft(nn.Module):
    def __init__(self):
        super(DenseNet121EyeLeft, self).__init__()
        self.backbone = models.densenet121(pretrained=True)
        self.backbone.classifier = nn.Identity()
        self.fc1 = nn.Linear(1024, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, len(ojo_izq) * 2)
        
    def forward(self, x):
        x = self.backbone(x)
        x = nn.ReLU()(self.fc1(x))
        x = nn.ReLU()(self.fc2(x))
        return self.fc3(x)

# Configuración y entrenamiento
model = DenseNet121EyeLeft().cuda()
optimizer = optim.AdamW(model.parameters(), lr=1e-4)
criterion = nn.MSELoss()
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=75, verbose=True)

def calculate_nme(predictions, ground_truths, num_landmarks):
    """
    Calcula el NME (Normalized Mean Error) utilizando la fórmula específica.
    
    :param predictions: Array de predicciones, de tamaño (N, M, 2), donde
                        N es el número de imágenes, M el número de landmarks, 
                        y 2 son las coordenadas (x, y) de cada punto.
    :param ground_truths: Array de valores reales (ground truths), del mismo tamaño que predictions.
    :param num_landmarks: Número de landmarks en cada imagen (M).
    :return: NME calculado sobre todo el dataset.
    """
    num_images = predictions.shape[0]
    total_error = 0.0

    for i in range(num_images):
        # Verificar que los índices utilizados para calcular la distancia interocular sean correctos
        if len(ground_truths[i]) < 5:
            raise ValueError(f"Expected at least 5 landmarks, but got {len(ground_truths[i])}")

        # Calcula la distancia interocular usando los puntos 0 y 4 (ajustado para ojo izquierdo)
        iodi = np.linalg.norm(ground_truths[i, 0] - ground_truths[i, 4])
        
        # Suma de errores normalizados por imagen
        for j in range(num_landmarks):
            pred_coord = predictions[i, j]
            gt_coord = ground_truths[i, j]
            error = np.linalg.norm(pred_coord - gt_coord) / iodi
            total_error += error
    
    # Aplicar la fórmula para el NME
    nme = total_error / (num_landmarks * num_images)
    return nme


def train_model(model, train_dataloader, optimizer, criterion, num_epochs=300, val_dataloader=None, scheduler=None):
    best_loss = float('inf')
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for images, eye_left_points, _, _, _ in train_dataloader:
            images, eye_left_points = images.cuda(), eye_left_points.cuda()
            outputs = model(images)
            loss = criterion(outputs, eye_left_points)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        
        epoch_loss = running_loss / len(train_dataloader)
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}')
        
        # Validación y cálculo del NME
        if val_dataloader:
            model.eval()
            val_loss = 0.0
            all_labels = []
            all_predictions = []
            with torch.no_grad():
                for val_images, val_eye_left_points, _, _, _ in val_dataloader:
                    val_images, val_eye_left_points = val_images.cuda(), val_eye_left_points.cuda()
                    val_outputs = model(val_images)
                    val_loss += criterion(val_outputs, val_eye_left_points).item()
                    
                    # Guardar las predicciones y etiquetas reales para el cálculo de NME
                    all_labels.append(val_eye_left_points.cpu().numpy().reshape(-1, len(ojo_izq), 2))
                    all_predictions.append(val_outputs.cpu().numpy().reshape(-1, len(ojo_izq), 2))

            val_loss /= len(val_dataloader)
            print(f'Validation Loss: {val_loss:.4f}')
            
            # Concatenar todas las predicciones y *ground truths* para el NME
            all_labels = np.concatenate(all_labels, axis=0)
            all_predictions = np.concatenate(all_predictions, axis=0)
            nme = calculate_nme(all_predictions, all_labels, num_landmarks=len(ojo_izq))
            print(f'NME: {nme:.4f}')
            
            # Guardar el mejor modelo
            if val_loss < best_loss:
                best_loss = val_loss
                torch.save(model.state_dict(), 'EyeLeftLandmarks V4.pth')
                print(f'Best model saved at epoch {epoch+1} with val_loss {best_loss:.4f}')
                
            if scheduler:
                scheduler.step(val_loss)

    print("Training complete.")

# Entrenar el modelo
train_model(model, train_dataloader, optimizer, criterion, num_epochs=300, val_dataloader=val_dataloader, scheduler=scheduler)

c:\Users\luisb250\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\luisb250\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet121_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet121_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
c:\Users\luisb250\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\optim\lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Epoch [1/300], Loss: 14416.8105
Validation Loss: 12798.8743
NME: 6.9950
Best model saved at epoch 1 with val_loss 12798.8743
Epoch [2/300], Loss: 8507.8556
Validation Loss: 3531.4210
NME: 3.3208
Best model saved at epoch 2 with val_loss 3531.4210
Epoch [3/300], Loss: 1677.0004
Validation Loss: 845.0300
NME: 1.1346
Best model saved at epoch 3 with val_loss 845.0300
Epoch [4/300], Loss: 721.7680
Validation Loss: 595.2252
NME: 0.9572
Best model saved at epoch 4 with val_loss 595.2252
Epoch [5/300], Loss: 626.4071
Validation Loss: 572.0977
NME: 1.0285
Best model saved at epoch 5 with val_loss 572.0977
Epoch [6/300], Loss: 618.7731
Validation Loss: 561.6694
NME: 1.0372
Best model saved at epoch 6 with val_loss 561.6694
Epoch [7/300], Loss: 617.4756
Validation Loss: 558.6201
NME: 1.0241
Best model saved at epoch 7 with val_loss 558.6201
Epoch [8/300], Loss: 614.1722
Validation Loss: 556.6554
NME: 1.0366
Best model saved at epoch 8 with val_loss 556.6554
Epoch [9/300], Loss: 614.4481
Validati

In [4]:
import os
import json
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from torch import nn, optim
from torchvision import models
from PIL import Image
import albumentations as A
from albumentations.pytorch import ToTensorV2
import cv2

# Rutas
train_img_dir = r'C:\CatFLW dataset\train\images'
train_labels_dir = r'C:\CatFLW dataset\train\labels'
val_img_dir = r'C:\CatFLW dataset\val\images'
val_labels_dir = r'C:\CatFLW dataset\val\labels'

# Definir los índices de las regiones según los puntos proporcionados
ojo_der = [3, 4, 5, 6, 7, 36, 37, 38]
ojo_izq = [1, 8, 9, 10, 11, 39, 40, 41]

# Función para calcular el punto promedio de una región
def calcular_centro_region(landmarks, indices):
    puntos = [landmarks[i] for i in indices]
    centro = np.mean(puntos, axis=0)
    return centro

# Dataset personalizado
class EyeRightDataset(Dataset):
    def __init__(self, img_dir, labels_dir, transform=None):
        self.img_dir = img_dir
        self.labels_dir = labels_dir
        self.transform = transform
        self.image_files = [f for f in os.listdir(img_dir) if f.endswith('.png')]

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_name = self.image_files[idx]
        img_path = os.path.join(self.img_dir, img_name)
        label_path = os.path.join(self.labels_dir, img_name.replace('.png', '.json'))
        
        # Cargar imagen
        image = np.array(Image.open(img_path).convert('RGB'))
        
        # Cargar etiquetas y puntos de referencia
        with open(label_path, 'r') as f:
            labels = json.load(f)
            landmarks = np.array(labels['labels'])  # Convertir a array de NumPy
            bounding_box = labels['bounding_boxes']
        
        # Recortar la imagen usando el bounding box
        x_min, y_min, x_max, y_max = map(int, bounding_box)
        image_cropped = image[y_min:y_max, x_min:x_max]
        
        # Calcular dimensiones originales
        original_height, original_width = image_cropped.shape[:2]
        
        # Calcular centros de los ojos
        centro_ojo_der = calcular_centro_region(landmarks, ojo_der)
        centro_ojo_izq = calcular_centro_region(landmarks, ojo_izq)
        
        # Ajustar coordenadas de los centros al recorte
        centro_ojo_der -= np.array([x_min, y_min])
        centro_ojo_izq -= np.array([x_min, y_min])
        
        # Calcular el ángulo de rotación
        dx = centro_ojo_izq[0] - centro_ojo_der[0]
        dy = centro_ojo_izq[1] - centro_ojo_der[1]
        angle = np.degrees(np.arctan2(dy, dx))
        
        # Crear matriz de rotación
        eye_center = ((centro_ojo_der[0] + centro_ojo_izq[0]) // 2,
                      (centro_ojo_der[1] + centro_ojo_izq[1]) // 2)
        rotation_matrix = cv2.getRotationMatrix2D(eye_center, angle, 1.0)
        
        # Rotar la imagen
        image_aligned = cv2.warpAffine(
            image_cropped, rotation_matrix, (original_width, original_height), flags=cv2.INTER_LINEAR)
        
        # Convertir los puntos clave del ojo derecho
        eye_right_points = np.array([landmarks[i] for i in ojo_der], dtype=np.float32)
        eye_right_points -= [x_min, y_min]

        # Transformar los puntos clave con la misma rotación
        ones = np.ones(shape=(len(eye_right_points), 1))
        points_ones = np.hstack([eye_right_points, ones])
        eye_right_points_rotated = rotation_matrix.dot(points_ones.T).T
        
        # **Nuevo**: Recorte alrededor del punto centrado del ojo derecho
        centro_x, centro_y = calcular_centro_region(eye_right_points_rotated, range(len(ojo_der)))

        # Definir los límites del recorte de 56x56 píxeles
        half_crop_size = 28
        x1 = int(max(0, centro_x - half_crop_size))
        y1 = int(max(0, centro_y - half_crop_size))
        x2 = int(min(original_width, centro_x + half_crop_size))
        y2 = int(min(original_height, centro_y + half_crop_size))
        
        # Recortar la imagen
        image_cropped_eye = image_aligned[y1:y2, x1:x2]
        
        # Redimensionar a 224x224
        image_resized = cv2.resize(image_cropped_eye, (224, 224))

        # Ajustar los puntos faciales del ojo derecho al nuevo recorte
        eye_right_points_adjusted = [(p[0] - x1, p[1] - y1) for p in eye_right_points_rotated]

        # Redimensionar las coordenadas de los puntos a la escala 224x224
        scale_x = 224 / (x2 - x1)
        scale_y = 224 / (y2 - y1)
        eye_right_points_scaled = [(p[0] * scale_x, p[1] * scale_y) for p in eye_right_points_adjusted]

        # Aplicar transformaciones de imagen y etiquetas
        if self.transform:
            augmented = self.transform(image=image_resized, keypoints=eye_right_points_scaled)
            image = augmented['image']
            eye_right_points = np.array(augmented['keypoints'], dtype=np.float32).flatten()
        
        return image, torch.tensor(eye_right_points).float(), img_name, original_width, original_height

# Transformaciones de aumento de datos para el conjunto de entrenamiento
train_transforms = A.Compose([
    A.Rotate(limit=30, p=0.5),  # Rotación aleatoria con 50% de probabilidad
    A.ColorJitter(p=0.5),  # Cambio en el balance de color, brillo, contraste y nitidez con 50% de probabilidad
    A.RandomBrightnessContrast(p=0.5),  # Cambio en el brillo y contraste con 50% de probabilidad
    A.GaussianBlur(p=0.1),  # Aplicación de máscaras de desenfoque con 50% de probabilidad
    A.GaussNoise(p=0.1),  # Ruido aleatorio con 50% de probabilidad
    A.Resize(224, 224),  # Redimensionar a 224x224
    A.Normalize(),
    ToTensorV2(),
], keypoint_params=A.KeypointParams(format='xy', remove_invisible=False))

# Transformaciones para el conjunto de validación
val_transforms = A.Compose([
    A.Resize(224, 224),  # Redimensionar a 224x224
    A.Normalize(),
    ToTensorV2(),
], keypoint_params=A.KeypointParams(format='xy', remove_invisible=False))

# Crear los datasets de entrenamiento y validación
train_dataset = EyeRightDataset(train_img_dir, train_labels_dir, transform=train_transforms)
val_dataset = EyeRightDataset(val_img_dir, val_labels_dir, transform=val_transforms)

# Crear DataLoaders
train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=16, shuffle=False)

# Modelo basado en DenseNet-121
class DenseNet121EyeRight(nn.Module):
    def __init__(self):
        super(DenseNet121EyeRight, self).__init__()
        self.backbone = models.densenet121(pretrained=True)
        self.backbone.classifier = nn.Identity()
        self.fc1 = nn.Linear(1024, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, len(ojo_der) * 2)
        
    def forward(self, x):
        x = self.backbone(x)
        x = nn.ReLU()(self.fc1(x))
        x = nn.ReLU()(self.fc2(x))
        return self.fc3(x)

# Configuración y entrenamiento
model = DenseNet121EyeRight().cuda()
optimizer = optim.AdamW(model.parameters(), lr=1e-4)
criterion = nn.MSELoss()
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=75, verbose=True)

def calculate_nme(predictions, ground_truths, num_landmarks):
    """
    Calcula el NME (Normalized Mean Error) utilizando la fórmula específica.
    
    :param predictions: Array de predicciones, de tamaño (N, M, 2), donde
                        N es el número de imágenes, M el número de landmarks, 
                        y 2 son las coordenadas (x, y) de cada punto.
    :param ground_truths: Array de valores reales (ground truths), del mismo tamaño que predictions.
    :param num_landmarks: Número de landmarks en cada imagen (M).
    :return: NME calculado sobre todo el dataset.
    """
    num_images = predictions.shape[0]
    total_error = 0.0

    for i in range(num_images):
        # Verificar que los índices utilizados para calcular la distancia interocular sean correctos
        if len(ground_truths[i]) < 5:
            raise ValueError(f"Expected at least 5 landmarks, but got {len(ground_truths[i])}")

        # Calcula la distancia interocular usando los puntos 0 y 4 (ajustado para ojo derecho)
        iodi = np.linalg.norm(ground_truths[i, 0] - ground_truths[i, 4])
        
        # Suma de errores normalizados por imagen
        for j in range(num_landmarks):
            pred_coord = predictions[i, j]
            gt_coord = ground_truths[i, j]
            error = np.linalg.norm(pred_coord - gt_coord) / iodi
            total_error += error
    
    # Aplicar la fórmula para el NME
    nme = total_error / (num_landmarks * num_images)
    return nme


def train_model(model, train_dataloader, optimizer, criterion, num_epochs=300, val_dataloader=None, scheduler=None):
    best_loss = float('inf')
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for images, eye_right_points, _, _, _ in train_dataloader:
            images, eye_right_points = images.cuda(), eye_right_points.cuda()
            outputs = model(images)
            loss = criterion(outputs, eye_right_points)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        
        epoch_loss = running_loss / len(train_dataloader)
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}')
        
        # Validación y cálculo del NME
        if val_dataloader:
            model.eval()
            val_loss = 0.0
            all_labels = []
            all_predictions = []
            with torch.no_grad():
                for val_images, val_eye_right_points, _, _, _ in val_dataloader:
                    val_images, val_eye_right_points = val_images.cuda(), val_eye_right_points.cuda()
                    val_outputs = model(val_images)
                    val_loss += criterion(val_outputs, val_eye_right_points).item()
                    
                    # Guardar las predicciones y etiquetas reales para el cálculo de NME
                    all_labels.append(val_eye_right_points.cpu().numpy().reshape(-1, len(ojo_der), 2))
                    all_predictions.append(val_outputs.cpu().numpy().reshape(-1, len(ojo_der), 2))

            val_loss /= len(val_dataloader)
            print(f'Validation Loss: {val_loss:.4f}')
            
            # Concatenar todas las predicciones y *ground truths* para el NME
            all_labels = np.concatenate(all_labels, axis=0)
            all_predictions = np.concatenate(all_predictions, axis=0)
            nme = calculate_nme(all_predictions, all_labels, num_landmarks=len(ojo_der))
            print(f'NME: {nme:.4f}')
            
            # Guardar el mejor modelo
            if val_loss < best_loss:
                best_loss = val_loss
                torch.save(model.state_dict(), 'EyeRightLandmarks V4.pth')
                print(f'Best model saved at epoch {epoch+1} with val_loss {best_loss:.4f}')
                
            if scheduler:
                scheduler.step(val_loss)

    print("Training complete.")

# Entrenar el modelo
train_model(model, train_dataloader, optimizer, criterion, num_epochs=300, val_dataloader=val_dataloader, scheduler=scheduler)

c:\Users\luisb250\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\luisb250\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet121_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet121_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
c:\Users\luisb250\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\optim\lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Epoch [1/300], Loss: 14571.5090
Validation Loss: 13211.5468
NME: 7.0548
Best model saved at epoch 1 with val_loss 13211.5468
Epoch [2/300], Loss: 9226.8351
Validation Loss: 4418.2021
NME: 3.6847
Best model saved at epoch 2 with val_loss 4418.2021
Epoch [3/300], Loss: 2134.2458
Validation Loss: 1037.6224
NME: 1.2995
Best model saved at epoch 3 with val_loss 1037.6224
Epoch [4/300], Loss: 865.8629
Validation Loss: 657.7015
NME: 1.0058
Best model saved at epoch 4 with val_loss 657.7015
Epoch [5/300], Loss: 677.6547
Validation Loss: 589.8464
NME: 1.0135
Best model saved at epoch 5 with val_loss 589.8464
Epoch [6/300], Loss: 646.9063
Validation Loss: 566.6760
NME: 1.0232
Best model saved at epoch 6 with val_loss 566.6760
Epoch [7/300], Loss: 630.8436
Validation Loss: 554.2906
NME: 1.0097
Best model saved at epoch 7 with val_loss 554.2906
Epoch [8/300], Loss: 631.0438
Validation Loss: 555.5191
NME: 1.0194
Epoch [9/300], Loss: 617.0227
Validation Loss: 542.5481
NME: 1.0172
Best model saved at

In [5]:
import os
import json
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader, random_split
from torch import nn, optim
from torchvision import models
from PIL import Image
import albumentations as A
from albumentations.pytorch import ToTensorV2
import cv2

# Rutas
train_img_dir = r'C:\CatFLW dataset\train\images'
train_labels_dir = r'C:\CatFLW dataset\train\labels'
val_img_dir = r'C:\CatFLW dataset\val\images'
val_labels_dir = r'C:\CatFLW dataset\val\labels'

# Definir los índices de las regiones según los puntos proporcionados
ojo_der = [3, 4, 5, 6, 7, 36, 37, 38]
ojo_izq = [1, 8, 9, 10, 11, 39, 40, 41]
nariz = [0, 2, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 32, 33, 34, 35, 42, 43, 44, 45, 46, 47]

# Función para calcular el punto promedio de una región
def calcular_centro_region(landmarks, indices):
    puntos = [landmarks[i] for i in indices]
    centro = np.mean(puntos, axis=0)
    return centro

# Dataset personalizado
class NoseDataset(Dataset):
    def __init__(self, img_dir, labels_dir, transform=None):
        self.img_dir = img_dir
        self.labels_dir = labels_dir
        self.transform = transform
        self.image_files = [f for f in os.listdir(img_dir) if f.endswith('.png')]

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_name = self.image_files[idx]
        img_path = os.path.join(self.img_dir, img_name)
        label_path = os.path.join(self.labels_dir, img_name.replace('.png', '.json'))
        
        # Cargar imagen
        image = np.array(Image.open(img_path).convert('RGB'))
        
        # Cargar etiquetas y puntos de referencia
        with open(label_path, 'r') as f:
            labels = json.load(f)
            landmarks = np.array(labels['labels'])  # Convertir a array de NumPy
            bounding_box = labels['bounding_boxes']
        
        # Recortar la imagen usando el bounding box
        x_min, y_min, x_max, y_max = map(int, bounding_box)
        image_cropped = image[y_min:y_max, x_min:x_max]
        
        # Calcular dimensiones originales
        original_height, original_width = image_cropped.shape[:2]
        
        # Calcular centros de los ojos y la nariz
        centro_ojo_der = calcular_centro_region(landmarks, ojo_der)
        centro_ojo_izq = calcular_centro_region(landmarks, ojo_izq)
        centro_nariz = calcular_centro_region(landmarks, nariz)
        
        # Ajustar coordenadas de los centros al recorte
        centro_ojo_der -= np.array([x_min, y_min])
        centro_ojo_izq -= np.array([x_min, y_min])
        centro_nariz -= np.array([x_min, y_min])
        
        # Calcular el ángulo de rotación
        dx = centro_ojo_izq[0] - centro_ojo_der[0]
        dy = centro_ojo_izq[1] - centro_ojo_der[1]
        angle = np.degrees(np.arctan2(dy, dx))
        
        # Crear matriz de rotación
        eye_center = ((centro_ojo_der[0] + centro_ojo_izq[0]) // 2,
                      (centro_ojo_der[1] + centro_ojo_izq[1]) // 2)
        rotation_matrix = cv2.getRotationMatrix2D(eye_center, angle, 1.0)
        
        # Rotar la imagen
        image_aligned = cv2.warpAffine(
            image_cropped, rotation_matrix, (original_width, original_height), flags=cv2.INTER_LINEAR)
        
        # Convertir los puntos clave de la nariz
        nose_points = np.array([landmarks[i] for i in nariz], dtype=np.float32)
        nose_points -= [x_min, y_min]

        # Transformar los puntos clave con la misma rotación
        ones = np.ones(shape=(len(nose_points), 1))
        points_ones = np.hstack([nose_points, ones])
        nose_points_rotated = rotation_matrix.dot(points_ones.T).T
        
        # **Nuevo**: Recorte alrededor del punto centrado de la nariz
        centro_x, centro_y = calcular_centro_region(nose_points_rotated, range(len(nariz)))

        # Definir los límites del recorte de 112x112 píxeles
        half_crop_size = 56
        x1 = int(max(0, centro_x - half_crop_size))
        y1 = int(max(0, centro_y - half_crop_size))
        x2 = int(min(original_width, centro_x + half_crop_size))
        y2 = int(min(original_height, centro_y + half_crop_size))
        
        # Recortar la imagen
        image_cropped_nose = image_aligned[y1:y2, x1:x2]
        
        # Redimensionar a 224x224
        image_resized = cv2.resize(image_cropped_nose, (224, 224))

        # Ajustar los puntos faciales de la nariz al nuevo recorte
        nose_points_adjusted = [(p[0] - x1, p[1] - y1) for p in nose_points_rotated]

        # Redimensionar las coordenadas de los puntos a la escala 224x224
        scale_x = 224 / (x2 - x1)
        scale_y = 224 / (y2 - y1)
        nose_points_scaled = [(p[0] * scale_x, p[1] * scale_y) for p in nose_points_adjusted]

        # Aplicar transformaciones de imagen y etiquetas
        if self.transform:
            augmented = self.transform(image=image_resized, keypoints=nose_points_scaled)
            image = augmented['image']
            nose_points = np.array(augmented['keypoints'], dtype=np.float32).flatten()
        
        return image, torch.tensor(nose_points).float(), img_name, original_width, original_height

# Transformaciones de aumento de datos para el conjunto de entrenamiento
train_transforms = A.Compose([
    A.Rotate(limit=30, p=0.5),  # Rotación aleatoria con 50% de probabilidad
    A.ColorJitter(p=0.5),  # Cambio en el balance de color, brillo, contraste y nitidez con 50% de probabilidad
    A.RandomBrightnessContrast(p=0.5),  # Cambio en el brillo y contraste con 50% de probabilidad
    A.GaussianBlur(p=0.1),  # Aplicación de máscaras de desenfoque con 50% de probabilidad
    A.GaussNoise(p=0.1),  # Ruido aleatorio con 50% de probabilidad
    A.Resize(224, 224),  # Redimensionar a 224x224
    A.Normalize(),
    ToTensorV2(),
], keypoint_params=A.KeypointParams(format='xy', remove_invisible=False))

# Transformaciones para el conjunto de validación
val_transforms = A.Compose([
    A.Resize(224, 224),  # Redimensionar a 224x224
    A.Normalize(),
    ToTensorV2(),
], keypoint_params=A.KeypointParams(format='xy', remove_invisible=False))

# Crear los datasets de entrenamiento y validación
train_dataset = NoseDataset(train_img_dir, train_labels_dir, transform=train_transforms)
val_dataset = NoseDataset(val_img_dir, val_labels_dir, transform=val_transforms)

# Crear DataLoaders
train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=16, shuffle=False)

# Modelo basado en DenseNet-121
class DenseNet121Nose(nn.Module):
    def __init__(self):
        super(DenseNet121Nose, self).__init__()
        self.backbone = models.densenet121(pretrained=True)
        self.backbone.classifier = nn.Identity()
        self.fc1 = nn.Linear(1024, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, len(nariz) * 2)
        
    def forward(self, x):
        x = self.backbone(x)
        x = nn.ReLU()(self.fc1(x))
        x = nn.ReLU()(self.fc2(x))
        return self.fc3(x)

# Configuración y entrenamiento
model = DenseNet121Nose().cuda()
optimizer = optim.AdamW(model.parameters(), lr=1e-4)
criterion = nn.MSELoss()
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=75, verbose=True)

def calculate_nme(predictions, ground_truths, num_landmarks):
    """
    Calcula el NME (Normalized Mean Error) utilizando la fórmula específica.
    
    :param predictions: Array de predicciones, de tamaño (N, M, 2), donde
                        N es el número de imágenes, M el número de landmarks, 
                        y 2 son las coordenadas (x, y) de cada punto.
    :param ground_truths: Array de valores reales (ground truths), del mismo tamaño que predictions.
    :param num_landmarks: Número de landmarks en cada imagen (M).
    :return: NME calculado sobre todo el dataset.
    """
    num_images = predictions.shape[0]
    total_error = 0.0

    for i in range(num_images):
        # Verificar que los índices utilizados para calcular la distancia interocular sean correctos
        if len(ground_truths[i]) < 5:
            raise ValueError(f"Expected at least 5 landmarks, but got {len(ground_truths[i])}")

        # Calcula la distancia interocular usando los puntos 0 y 4 (ajustado para nariz)
        iodi = np.linalg.norm(ground_truths[i, 0] - ground_truths[i, 4])
        
        # Suma de errores normalizados por imagen
        for j in range(num_landmarks):
            pred_coord = predictions[i, j]
            gt_coord = ground_truths[i, j]
            error = np.linalg.norm(pred_coord - gt_coord) / iodi
            total_error += error
    
    # Aplicar la fórmula para el NME
    nme = total_error / (num_landmarks * num_images)
    return nme


def train_model(model, train_dataloader, optimizer, criterion, num_epochs=300, val_dataloader=None, scheduler=None):
    best_loss = float('inf')
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for images, nose_points, _, _, _ in train_dataloader:
            images, nose_points = images.cuda(), nose_points.cuda()
            outputs = model(images)
            loss = criterion(outputs, nose_points)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        
        epoch_loss = running_loss / len(train_dataloader)
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}')
        
        # Validación y cálculo del NME
        if val_dataloader:
            model.eval()
            val_loss = 0.0
            all_labels = []
            all_predictions = []
            with torch.no_grad():
                for val_images, val_nose_points, _, _, _ in val_dataloader:
                    val_images, val_nose_points = val_images.cuda(), val_nose_points.cuda()
                    val_outputs = model(val_images)
                    val_loss += criterion(val_outputs, val_nose_points).item()
                    
                    # Guardar las predicciones y etiquetas reales para el cálculo de NME
                    all_labels.append(val_nose_points.cpu().numpy().reshape(-1, len(nariz), 2))
                    all_predictions.append(val_outputs.cpu().numpy().reshape(-1, len(nariz), 2))

            val_loss /= len(val_dataloader)
            print(f'Validation Loss: {val_loss:.4f}')
            
            # Concatenar todas las predicciones y *ground truths* para el NME
            all_labels = np.concatenate(all_labels, axis=0)
            all_predictions = np.concatenate(all_predictions, axis=0)
            nme = calculate_nme(all_predictions, all_labels, num_landmarks=len(nariz))
            print(f'NME: {nme:.4f}')
            
            # Guardar el mejor modelo
            if val_loss < best_loss:
                best_loss = val_loss
                torch.save(model.state_dict(), 'NoseAreaLandmarks V4.pth')
                print(f'Best model saved at epoch {epoch+1} with val_loss {best_loss:.4f}')
                
            if scheduler:
                scheduler.step(val_loss)

    print("Training complete.")

# Entrenar el modelo
train_model(model, train_dataloader, optimizer, criterion, num_epochs=300, val_dataloader=val_dataloader, scheduler=scheduler)

c:\Users\luisb250\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\luisb250\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet121_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet121_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
c:\Users\luisb250\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\optim\lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Epoch [1/300], Loss: 16322.7618
Validation Loss: 14718.1610
NME: 3.3054
Best model saved at epoch 1 with val_loss 14718.1610
Epoch [2/300], Loss: 9639.4928
Validation Loss: 3998.1435
NME: 1.4770
Best model saved at epoch 2 with val_loss 3998.1435
Epoch [3/300], Loss: 2025.5064
Validation Loss: 1166.2557
NME: 0.6302
Best model saved at epoch 3 with val_loss 1166.2557
Epoch [4/300], Loss: 1034.8240
Validation Loss: 899.3866
NME: 0.5494
Best model saved at epoch 4 with val_loss 899.3866
Epoch [5/300], Loss: 937.6587
Validation Loss: 862.7673
NME: 0.5445
Best model saved at epoch 5 with val_loss 862.7673
Epoch [6/300], Loss: 927.2605
Validation Loss: 852.6251
NME: 0.5440
Best model saved at epoch 6 with val_loss 852.6251
Epoch [7/300], Loss: 906.8241
Validation Loss: 843.2822
NME: 0.5400
Best model saved at epoch 7 with val_loss 843.2822
Epoch [8/300], Loss: 900.3095
Validation Loss: 837.0300
NME: 0.5363
Best model saved at epoch 8 with val_loss 837.0300
Epoch [9/300], Loss: 887.5826
Valid